In [8]:
import cv2
import numpy as np
import os
import time
import mediapipe as mp
import matplotlib
import matplotlib.pyplot as plt

mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

def mediapipe_detection(image,model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image, results  

def draw_landmarks(image, results):
#     mp_drawing.draw_landmarks(image,results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    
def draw_styled_landmarks(image, results):
#     mp_drawing.draw_landmarks(image,results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
#                               mp_drawing.DrawingSpec(color=(121,22,76), thickness = 2, circle_radius=4),
#                               mp_drawing.DrawingSpec(color=(121,44,250), thickness = 2, circle_radius=2)
#                              )
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(245,117,66), thickness = 2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(245,66,230), thickness = 2, circle_radius=2)
                             )
    
def extract_keypoints(results):
#     lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
#     return np.concatenate([lh, rh])
    return np.concatenate([rh])

# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data4') 

#totalvideo
totalvideo = 7

# Actions that we try to detect
actions = np.array(['A', 'B', 'C','D', 'E', 'F','G', 'H', 'I','J', 'K', 'L','M', 'N', 'O','P', 'Q', 'R','S', 'T', 'U','V', 'W', 'X','Y', 'Z'])

# 15 potongan video dalam 1 video abjad
no_sequences = 15

# per potongan video diambil 30 frame
sequence_length = 30

In [2]:
for action in actions: 
    for sequence in range(no_sequences * totalvideo - totalvideo + 1):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [4]:
counter = 0 
length2 = sequence_length
sequence = -1
countervidbaru = 0
counterseqvidbaru = 0
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
     for action in actions:
        sequence = -1
        countervidbaru=0
        counterseqvidbaru = 0
        for index in range(1,totalvideo+1):
            if countervidbaru == 1:
                sequence -= 1
            path = 'dataset/video/' + action + '/' + str(index) +'.webm'
            counter = 0
            length2 = sequence_length
            # Loop through sequences aka videos
            while True:
                sequence += 1
                if sequence != 0:
                    if countervidbaru == 1:
                        countervidbaru = 0
                        counterseqvidbaru += 1
                    else:
                        if (sequence + counterseqvidbaru) % no_sequences == 0:
                            countervidbaru = 1
                            break
                cap = cv2.VideoCapture(path)
                length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                counter += 6
                if length > sequence_length+counter:
                    length2 = sequence_length+counter
                else:
                    counter = 0
                    length2 = sequence_length+counter
                cap.set(cv2.CAP_PROP_POS_FRAMES, counter)
                print('Collecting frames for ' + action + ' dataset number '+ str(index)+' Video Number ' + str(sequence))
#                 print("counter = "+ str(counter))
#                 print("sequence_length = " + str(sequence_length))
#                 print("length = " + str(length))
    #             cap.set(counter,length2);
                # Loop through video length aka sequence length
                for frame_num in range(sequence_length):

                    # Read feed
                    ret, frame = cap.read()

                    # Make detections
                    image, results = mediapipe_detection(frame, holistic)

                    # Draw landmarks
                    draw_styled_landmarks(image, results)

                    if frame_num == 0: 
                        cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                        cv2.putText(image, 'Collecting frames for {} dataset number {} Video Number {}'.format(action, index, sequence), (15,12), 
                                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                        # Show to screen
                        cv2.imshow('OpenCV Feed', image)
                    else: 
                        cv2.putText(image, 'Collecting frames for {} dataset number {} Video Number {}'.format(action, index, sequence), (15,12), 
                                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                        # Show to screen
                        cv2.imshow('OpenCV Feed', image)


                    # NEW Export keypoints
                    keypoints = extract_keypoints(results)
                    npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                    np.save(npy_path, keypoints)

                    # Break gracefully
                    if cv2.waitKey(10) & 0xFF == ord('q'):
                        break

            cap.release()
            cv2.destroyAllWindows()

6. PREPROCESS DATA AND CREATE LABELS FEATURES

In [9]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [10]:
label_map = {label:num for num, label in enumerate(actions)}

In [11]:
label_map

{'A': 0,
 'B': 1,
 'C': 2,
 'D': 3,
 'E': 4,
 'F': 5,
 'G': 6,
 'H': 7,
 'I': 8,
 'J': 9,
 'K': 10,
 'L': 11,
 'M': 12,
 'N': 13,
 'O': 14,
 'P': 15,
 'Q': 16,
 'R': 17,
 'S': 18,
 'T': 19,
 'U': 20,
 'V': 21,
 'W': 22,
 'X': 23,
 'Y': 24,
 'Z': 25}

In [12]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences * totalvideo - totalvideo + 1):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [13]:
np.array(sequences).shape

(2574, 30, 63)

In [14]:
np.array(labels).shape

(2574,)

In [17]:
X = np.array(sequences)

In [18]:
X.shape #total 90 video, per video punya 30 frame, per frame punya 1662 titik/keypoints

(2574, 30, 63)

In [19]:
y = to_categorical(labels).astype(int)

In [20]:
y.shape

(2574, 26)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [22]:
y_test.shape

(129, 26)

7. BUILD AND TRAIN LSTM NEURAL NETWORK

In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [24]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [25]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,63)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [26]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [27]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
77/77 [==============================] - 4s 29ms/step - loss: 3.2576 - categorical_accuracy: 0.0358
Epoch 2/2000
77/77 [==============================] - 2s 23ms/step - loss: 3.2564 - categorical_accuracy: 0.0497
Epoch 3/2000
77/77 [==============================] - 2s 23ms/step - loss: 3.2655 - categorical_accuracy: 0.0476
Epoch 4/2000
77/77 [==============================] - 2s 23ms/step - loss: 3.2429 - categorical_accuracy: 0.0581
Epoch 5/2000
77/77 [==============================] - 2s 23ms/step - loss: 3.0269 - categorical_accuracy: 0.0787
Epoch 6/2000
77/77 [==============================] - 2s 23ms/step - loss: 2.7236 - categorical_accuracy: 0.0998
Epoch 7/2000
77/77 [==============================] - 2s 23ms/step - loss: 2.5260 - categorical_accuracy: 0.1478
Epoch 8/2000
77/77 [==============================] - 2s 23ms/step - loss: 2.4820 - categorical_accuracy: 0.1752
Epoch 9/2000
77/77 [==============================] - 2s 23ms/step - loss: 2.4105 - categorical_

77/77 [==============================] - 2s 23ms/step - loss: 0.2056 - categorical_accuracy: 0.9230
Epoch 73/2000
77/77 [==============================] - 2s 23ms/step - loss: 0.1792 - categorical_accuracy: 0.9386
Epoch 74/2000
77/77 [==============================] - 2s 23ms/step - loss: 0.3305 - categorical_accuracy: 0.8835
Epoch 75/2000
77/77 [==============================] - 2s 23ms/step - loss: 0.4183 - categorical_accuracy: 0.8615
Epoch 76/2000
77/77 [==============================] - 2s 23ms/step - loss: 0.2208 - categorical_accuracy: 0.9261
Epoch 77/2000
77/77 [==============================] - 2s 23ms/step - loss: 0.2466 - categorical_accuracy: 0.9142
Epoch 78/2000
77/77 [==============================] - 2s 23ms/step - loss: 0.4325 - categorical_accuracy: 0.8570 1s - loss: 0.5
Epoch 79/2000
77/77 [==============================] - 2s 23ms/step - loss: 0.1450 - categorical_accuracy: 0.9545 0s - loss: 0.1349 - catego
Epoch 80/2000
77/77 [==============================] - 2s 23

77/77 [==============================] - 2s 22ms/step - loss: 0.1050 - categorical_accuracy: 0.9676
Epoch 143/2000
77/77 [==============================] - 2s 22ms/step - loss: 0.2481 - categorical_accuracy: 0.9201
Epoch 144/2000
77/77 [==============================] - 2s 22ms/step - loss: 0.2754 - categorical_accuracy: 0.9085
Epoch 145/2000
77/77 [==============================] - 2s 23ms/step - loss: 0.1455 - categorical_accuracy: 0.9584
Epoch 146/2000
77/77 [==============================] - 2s 23ms/step - loss: 0.0595 - categorical_accuracy: 0.9856
Epoch 147/2000
77/77 [==============================] - 2s 23ms/step - loss: 0.0625 - categorical_accuracy: 0.9840 0s - loss: 0.0607 - categorical_accuracy: 
Epoch 148/2000
77/77 [==============================] - 2s 23ms/step - loss: 0.0645 - categorical_accuracy: 0.9797
Epoch 149/2000
77/77 [==============================] - 2s 22ms/step - loss: 0.0898 - categorical_accuracy: 0.9696 0s - loss: 0.0862 - catego
Epoch 150/2000
77/77 [===

77/77 [==============================] - 2s 23ms/step - loss: 0.0767 - categorical_accuracy: 0.9779
Epoch 212/2000
77/77 [==============================] - 2s 22ms/step - loss: 0.0819 - categorical_accuracy: 0.9765
Epoch 213/2000
77/77 [==============================] - 2s 22ms/step - loss: 0.1112 - categorical_accuracy: 0.9774
Epoch 214/2000
77/77 [==============================] - 2s 22ms/step - loss: 0.3101 - categorical_accuracy: 0.9016
Epoch 215/2000
77/77 [==============================] - 2s 22ms/step - loss: 0.1896 - categorical_accuracy: 0.9394
Epoch 216/2000
77/77 [==============================] - 2s 22ms/step - loss: 0.0811 - categorical_accuracy: 0.9756
Epoch 217/2000
77/77 [==============================] - 2s 22ms/step - loss: 0.0666 - categorical_accuracy: 0.9781
Epoch 218/2000
77/77 [==============================] - 2s 22ms/step - loss: 0.0392 - categorical_accuracy: 0.9903
Epoch 219/2000
77/77 [==============================] - 2s 23ms/step - loss: 0.0875 - categoric

KeyboardInterrupt: 

In [28]:
model.save('modeljadi7.h5')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 30, 64)            32768     
_________________________________________________________________
lstm_1 (LSTM)                (None, 30, 128)           98816     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 26)                858       
Total params: 188,090
Trainable params: 188,090
Non-trainable params: 0
__________________________________________________

In [8]:
model.load_weights('modeljadi7.h5')

8. MAKE PREDICTIONS

In [29]:
res = model.predict(X_test)

In [30]:
actions[np.argmax(res[4])]

'D'

In [31]:
actions[np.argmax(y_test[4])]

'D'

10. EVALUATION USING CONFUSION MATRIX AND ACCURACY

In [32]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [33]:
yhat = model.predict(X_test)

In [34]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [35]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[125,   0],
        [  0,   4]],

       [[125,   0],
        [  0,   4]],

       [[126,   0],
        [  0,   3]],

       [[118,   1],
        [  0,  10]],

       [[122,   1],
        [  0,   6]],

       [[125,   0],
        [  0,   4]],

       [[122,   0],
        [  1,   6]],

       [[119,   0],
        [  0,  10]],

       [[122,   0],
        [  2,   5]],

       [[123,   0],
        [  0,   6]],

       [[124,   0],
        [  0,   5]],

       [[124,   0],
        [  0,   5]],

       [[125,   0],
        [  0,   4]],

       [[126,   1],
        [  0,   2]],

       [[121,   1],
        [  0,   7]],

       [[122,   0],
        [  0,   7]],

       [[125,   0],
        [  0,   4]],

       [[124,   0],
        [  1,   4]],

       [[126,   0],
        [  0,   3]],

       [[127,   0],
        [  0,   2]],

       [[125,   1],
        [  0,   3]],

       [[121,   0],
        [  0,   8]],

       [[127,   0],
        [  0,   2]],

       [[123,   0],
        [  0, 

In [36]:
accuracy_score(ytrue, yhat)

0.9612403100775194

11. TEST IN REAL TIME

In [38]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [43]:
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.9

cap = cv2.VideoCapture(2)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.3, min_tracking_confidence=0.3) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            
            
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
R
<class 'mediapipe.python.solution_base.SolutionOutputs'>
R
<class 'mediapipe.python.solution_base.SolutionOutputs'>
R
<class 'mediapipe.python.solution_base.SolutionOutputs'>
R
<class 'mediapipe.python.solution_base.SolutionOutputs'>
R
<class 'mediapipe.python.solution_base.SolutionOutputs'>
R
<class 'mediapipe.python.solution_base.SolutionOutputs'>
R
<class 'mediapipe.python.solution_base.SolutionOutputs'>
R
<class 'mediapipe.python.solution_base.SolutionOutputs'>
R
<class 'mediapipe.python.solution_base.SolutionOutputs'>
R
<class 'mediapipe.python.solution_base.SolutionOutputs'>
R
<class 'mediapipe.python.solution_base.SolutionOutputs'>
R
<class 'mediapipe.python.solution_base.SolutionOutputs'>
R
<class 'mediapipe.python.solution_base.SolutionOutputs'>
R
<class 'mediapipe.python.solution_base.SolutionOutputs'>
R
<class 'mediapipe.python.solution_base.SolutionOutputs'>
B
<class 'mediapipe.python.solution_base.SolutionOutputs'>

<class 'mediapipe.python.solution_base.SolutionOutputs'>
S
<class 'mediapipe.python.solution_base.SolutionOutputs'>
S
<class 'mediapipe.python.solution_base.SolutionOutputs'>
S
<class 'mediapipe.python.solution_base.SolutionOutputs'>
S
<class 'mediapipe.python.solution_base.SolutionOutputs'>
S
<class 'mediapipe.python.solution_base.SolutionOutputs'>
S
<class 'mediapipe.python.solution_base.SolutionOutputs'>
S
<class 'mediapipe.python.solution_base.SolutionOutputs'>
S
<class 'mediapipe.python.solution_base.SolutionOutputs'>
S
<class 'mediapipe.python.solution_base.SolutionOutputs'>
S
<class 'mediapipe.python.solution_base.SolutionOutputs'>
S
<class 'mediapipe.python.solution_base.SolutionOutputs'>
S
<class 'mediapipe.python.solution_base.SolutionOutputs'>
S
<class 'mediapipe.python.solution_base.SolutionOutputs'>
S
<class 'mediapipe.python.solution_base.SolutionOutputs'>
S
<class 'mediapipe.python.solution_base.SolutionOutputs'>
S
<class 'mediapipe.python.solution_base.SolutionOutputs'>

<class 'mediapipe.python.solution_base.SolutionOutputs'>
F
<class 'mediapipe.python.solution_base.SolutionOutputs'>
F
<class 'mediapipe.python.solution_base.SolutionOutputs'>
F
<class 'mediapipe.python.solution_base.SolutionOutputs'>
F
<class 'mediapipe.python.solution_base.SolutionOutputs'>
F
<class 'mediapipe.python.solution_base.SolutionOutputs'>
F
<class 'mediapipe.python.solution_base.SolutionOutputs'>
F
<class 'mediapipe.python.solution_base.SolutionOutputs'>
F
<class 'mediapipe.python.solution_base.SolutionOutputs'>
F
<class 'mediapipe.python.solution_base.SolutionOutputs'>
F
<class 'mediapipe.python.solution_base.SolutionOutputs'>
F
<class 'mediapipe.python.solution_base.SolutionOutputs'>
S
<class 'mediapipe.python.solution_base.SolutionOutputs'>
S
<class 'mediapipe.python.solution_base.SolutionOutputs'>
S
<class 'mediapipe.python.solution_base.SolutionOutputs'>
S
<class 'mediapipe.python.solution_base.SolutionOutputs'>
S
<class 'mediapipe.python.solution_base.SolutionOutputs'>

<class 'mediapipe.python.solution_base.SolutionOutputs'>
W
<class 'mediapipe.python.solution_base.SolutionOutputs'>
W
<class 'mediapipe.python.solution_base.SolutionOutputs'>
W
<class 'mediapipe.python.solution_base.SolutionOutputs'>
W
<class 'mediapipe.python.solution_base.SolutionOutputs'>
W
<class 'mediapipe.python.solution_base.SolutionOutputs'>
W
<class 'mediapipe.python.solution_base.SolutionOutputs'>
W
<class 'mediapipe.python.solution_base.SolutionOutputs'>
W
<class 'mediapipe.python.solution_base.SolutionOutputs'>
W
<class 'mediapipe.python.solution_base.SolutionOutputs'>
W
<class 'mediapipe.python.solution_base.SolutionOutputs'>
W
<class 'mediapipe.python.solution_base.SolutionOutputs'>
W
<class 'mediapipe.python.solution_base.SolutionOutputs'>
W
<class 'mediapipe.python.solution_base.SolutionOutputs'>
W
<class 'mediapipe.python.solution_base.SolutionOutputs'>
W
<class 'mediapipe.python.solution_base.SolutionOutputs'>
W
<class 'mediapipe.python.solution_base.SolutionOutputs'>

...

<class 'mediapipe.python.solution_base.SolutionOutputs'>
K
<class 'mediapipe.python.solution_base.SolutionOutputs'>
K
<class 'mediapipe.python.solution_base.SolutionOutputs'>
K
<class 'mediapipe.python.solution_base.SolutionOutputs'>
K
<class 'mediapipe.python.solution_base.SolutionOutputs'>
K
<class 'mediapipe.python.solution_base.SolutionOutputs'>
K
<class 'mediapipe.python.solution_base.SolutionOutputs'>
K
<class 'mediapipe.python.solution_base.SolutionOutputs'>
K
<class 'mediapipe.python.solution_base.SolutionOutputs'>
K
<class 'mediapipe.python.solution_base.SolutionOutputs'>
K
<class 'mediapipe.python.solution_base.SolutionOutputs'>
K
<class 'mediapipe.python.solution_base.SolutionOutputs'>
K
<class 'mediapipe.python.solution_base.SolutionOutputs'>
K
<class 'mediapipe.python.solution_base.SolutionOutputs'>
K
<class 'mediapipe.python.solution_base.SolutionOutputs'>
K
<class 'mediapipe.python.solution_base.SolutionOutputs'>
K
<class 'mediapipe.python.solution_base.SolutionOutputs'>

<class 'mediapipe.python.solution_base.SolutionOutputs'>
K
<class 'mediapipe.python.solution_base.SolutionOutputs'>
K
<class 'mediapipe.python.solution_base.SolutionOutputs'>
K
<class 'mediapipe.python.solution_base.SolutionOutputs'>
K
<class 'mediapipe.python.solution_base.SolutionOutputs'>
K
<class 'mediapipe.python.solution_base.SolutionOutputs'>
K
<class 'mediapipe.python.solution_base.SolutionOutputs'>
K
<class 'mediapipe.python.solution_base.SolutionOutputs'>
K
<class 'mediapipe.python.solution_base.SolutionOutputs'>
K
<class 'mediapipe.python.solution_base.SolutionOutputs'>
K
<class 'mediapipe.python.solution_base.SolutionOutputs'>
K
<class 'mediapipe.python.solution_base.SolutionOutputs'>
K
<class 'mediapipe.python.solution_base.SolutionOutputs'>
K
<class 'mediapipe.python.solution_base.SolutionOutputs'>
K
<class 'mediapipe.python.solution_base.SolutionOutputs'>
K
<class 'mediapipe.python.solution_base.SolutionOutputs'>
K
<class 'mediapipe.python.solution_base.SolutionOutputs'>

<class 'mediapipe.python.solution_base.SolutionOutputs'>
W
<class 'mediapipe.python.solution_base.SolutionOutputs'>
W
<class 'mediapipe.python.solution_base.SolutionOutputs'>
W
<class 'mediapipe.python.solution_base.SolutionOutputs'>
W
<class 'mediapipe.python.solution_base.SolutionOutputs'>
W
<class 'mediapipe.python.solution_base.SolutionOutputs'>
W
<class 'mediapipe.python.solution_base.SolutionOutputs'>
W
<class 'mediapipe.python.solution_base.SolutionOutputs'>
W
<class 'mediapipe.python.solution_base.SolutionOutputs'>
W
<class 'mediapipe.python.solution_base.SolutionOutputs'>
W
<class 'mediapipe.python.solution_base.SolutionOutputs'>
W
<class 'mediapipe.python.solution_base.SolutionOutputs'>
W
<class 'mediapipe.python.solution_base.SolutionOutputs'>
W
<class 'mediapipe.python.solution_base.SolutionOutputs'>
W
<class 'mediapipe.python.solution_base.SolutionOutputs'>
W
<class 'mediapipe.python.solution_base.SolutionOutputs'>
W
<class 'mediapipe.python.solution_base.SolutionOutputs'>

<class 'mediapipe.python.solution_base.SolutionOutputs'>
Y
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Y
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Y
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Y
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Y
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Y
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Y
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Y
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Y
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Y
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Y
<class 'mediapipe.python.solution_base.SolutionOutputs'>
J
<class 'mediapipe.python.solution_base.SolutionOutputs'>
J
<class 'mediapipe.python.solution_base.SolutionOutputs'>
J
<class 'mediapipe.python.solution_base.SolutionOutputs'>
J
<class 'mediapipe.python.solution_base.SolutionOutputs'>
J
<class 'mediapipe.python.solution_base.SolutionOutputs'>

<class 'mediapipe.python.solution_base.SolutionOutputs'>
Z
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Z
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Z
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Z
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Z
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Z
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Z
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Z
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Q
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Q
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Q
<class 'mediapipe.python.solution_base.SolutionOutputs'>
X
<class 'mediapipe.python.solution_base.SolutionOutputs'>
X
<class 'mediapipe.python.solution_base.SolutionOutputs'>
X
<class 'mediapipe.python.solution_base.SolutionOutputs'>
X
<class 'mediapipe.python.solution_base.SolutionOutputs'>
X
<class 'mediapipe.python.solution_base.SolutionOutputs'>